# Motor de recomendación con PySpark

___
## Tarea 1 - Módulo 8

La práctica consiste en construir un recomendador con el módulo de recomendación de Spark (en la versión 1.6 solamente implementa el método ALS) sobre un conjunto de entrenamiento, realizar predicciones sobre un conjunto de test y posteriormente evaluar su rendimiento aplicando un RegressionEvaluator con metricName=”rmse”, dado que los valores finales son las valoraciones que se supone que daría el usuario a una determinada película que aún no ha visto, y por tanto numéricos.

El paquete de pySpark de recomendaciones se encuentra en:

http://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS

El recomendador se construirá utilizando el dataset de 100000 puntuaciones de 1000 usuarios sobre 1700 películas de Movielens

http://grouplens.org/datasets/movielens/100k/  (user id | item id | rating | timestamp)

Se recomienda leer el archivo README.txt para entender bien los datos:

http://files.grouplens.org/datasets/movielens/ml-100k-README.txt


El entregable será un notebook de Python con el código pyspark correspondiente.

___
## Preparación del entorno: paquetes y variables

Lo primero que vamos hacer es **importar todos los paquetes** necesarios y crear el contexto para los data frames (SQLContext)

In [4]:
import re
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.recommendation import ALS, ALSModel

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

sqlc = SQLContext(sc)

A continuación se crean las **variables** para:
* Las rutas y nombres de los siguientes ficheros: puntuaciones usuario-película, información de los usuarios, información de las películas, información de las categorías de las películas
* El separador de los campos en los ficheros que pueden ser un pipe o un tabulador
* El nombre y posición de las columnas de los ficheros; además del tipo de dato que será en el dataframe

In [6]:
# Rutas, nombres de los datasets y separadores que se usan

RUTA_FICHERO_PUNTUACION = '/FileStore/tables/u.data'
RUTA_FICHERO_USUARIO = '/FileStore/tables/u.user'
RUTA_FICHERO_ITEM = '/FileStore/tables/u.item'
RUTA_FICHERO_GENERO = '/FileStore/tables/u.genre'
SEPARADOR_TABULADOR = '\t'
SEPARADOR_PIPE = '|'

# Variables con información de las columnas de los ficheros: 
#    - posicion 0: nombre de la columna en el fichero
#    - posicion 1: posicion de la columna en el fichero
#    - posicion 2: tipo dataframe
#    - posicion 3: True si la columna puede ser nullable. Se utilizara al convertir de RDD a dataframe.
# Se crea también una lista con ellos dentro por si se quiere iterar


# Dataset de puntuaciones

COL_PUNTUACION_USERID = ('userid', 0, IntegerType(), True)
COL_PUNTUACION_ITEMID = ('itemid', 1, IntegerType(), True)
COL_PUNTUACION_RATING = ('rating', 2, FloatType(), True)
COL_PUNTUACION_TIMESTAMP = ('timestamp', 3, IntegerType(), True)
COL_PUNTUACION_PREDICTION = ('prediction', None, None, None)
COL_PUNTUACION_NUMEROPUNTUACIONES = ('numero_puntuaciones', None, None, None)
COLS_PUNTUACION = (
    COL_PUNTUACION_USERID, 
    COL_PUNTUACION_ITEMID, 
    COL_PUNTUACION_RATING, 
    COL_PUNTUACION_TIMESTAMP)

# Dataset de usuarios

COL_USUARIO_ID = ('user_userid', 0, IntegerType(), True)
COL_USUARIO_AGE = ('age', 1, IntegerType(), True)
COL_USUARIO_GENDER = ('gender', 2, StringType(), True)
COL_USUARIO_OCCUPATION = ('occupation', 3, StringType(), True)
COL_USUARIO_ZIPCODE = ('zipcode', 4, StringType(), True)
COLS_USUARIO = (
    COL_USUARIO_ID, 
    COL_USUARIO_AGE, 
    COL_USUARIO_GENDER, 
    COL_USUARIO_OCCUPATION,
    COL_USUARIO_ZIPCODE)

#  Dataset de generos

COL_GENERO_NAME = ('name', 0, StringType(), False)
COL_GENERO_ID = ('genre_genreid', 1, IntegerType(), False)
COLS_GENERO = (
    COL_GENERO_NAME, 
    COL_GENERO_ID)

#  Dataset de items

COL_ITEM_ID = ('item_itemid', 0, IntegerType(), False)
COL_ITEM_TITLE = ('title', 1, StringType(), False)
COL_ITEM_RELEASEDATE = ('releasedate', 2, StringType(), False)
COL_ITEM_VIDERELEASEDATE = ('videoreleasedate', 3, StringType(), False)
COL_ITEM_IMDBURL = ('imdburl', 4, StringType(), False)
COL_ITEM_UNKNOWN = ('unknown', 5, IntegerType(), False)
COL_ITEM_ACTION = ('action', 6, IntegerType(), False)
COL_ITEM_ADVENTURE = ('adventure', 7, IntegerType(), False)
COL_ITEM_ANIMATION = ('animation', 8, IntegerType(), False)
COL_ITEM_CHILDRENS = ('childrens', 9, IntegerType(), False)
COL_ITEM_COMEDY = ('comedy', 10, IntegerType(), False)
COL_ITEM_CRIME = ('crime', 11, IntegerType(), False)
COL_ITEM_DOCUMENTARY = ('documentary', 12, IntegerType(), False)
COL_ITEM_DRAMA = ('drama', 13, IntegerType(), False)
COL_ITEM_FANTASY = ('fantasy', 14, IntegerType(), False)
COL_ITEM_FILMNOIR = ('filmnoir', 15, IntegerType(), False)
COL_ITEM_HORROR = ('horror', 16, IntegerType(), False)
COL_ITEM_MUSICAL = ('musical', 17, IntegerType(), False)
COL_ITEM_MYSTERY = ('mystery', 18, IntegerType(), False)
COL_ITEM_ROMACE = ('Romance', 19, IntegerType(), False)
COL_ITEM_SCIFI = ('scifi', 20, IntegerType(), False)
COL_ITEM_THRILLER = ('thriller', 21, IntegerType(), False)
COL_ITEM_WAR = ('war', 22, IntegerType(), False)
COL_ITEM_WESTERN = ('western', 23, IntegerType(), False)
COL_ITEM_GENEROS = ('generos', None, ArrayType(StringType()), False)
COL_ITEM_GENERO = ('genero', None, None, None)
COLS_ITEM = (
    COL_ITEM_ID, 
    COL_ITEM_TITLE,
    COL_ITEM_RELEASEDATE,
    COL_ITEM_VIDERELEASEDATE,
    COL_ITEM_IMDBURL)
COLS_ITEM_TODOSGENEROS = (
    COL_ITEM_UNKNOWN,
    COL_ITEM_ACTION, 
    COL_ITEM_ADVENTURE,
    COL_ITEM_ANIMATION,
    COL_ITEM_CHILDRENS,
    COL_ITEM_COMEDY,
    COL_ITEM_CRIME,
    COL_ITEM_DOCUMENTARY,
    COL_ITEM_DRAMA,
    COL_ITEM_FANTASY,
    COL_ITEM_FILMNOIR,
    COL_ITEM_HORROR,
    COL_ITEM_MUSICAL,
    COL_ITEM_MYSTERY,
    COL_ITEM_ROMACE,
    COL_ITEM_SCIFI,
    COL_ITEM_THRILLER,
    COL_ITEM_WAR,
    COL_ITEM_WESTERN)


___
## Creación de los dataframes

Creamos los dataframes con las siguientes particularidades:

* **Puntuaciones**: Se elimina el atributo timestamp, ya que no se cree que aporte nada al modelo que queremos construir.

* **Item**: Se eliminarán los atributos video_release_date e IMDb_URL, por la misma razón que anteriormente.

Finalmente uniremos todos los dataframes en uno solo para hacer la modelización.

In [8]:
# Se carga las puntuaciones utilizando COLS_PUNTUACION y después se elimina el timestamp
esquemaPuntuacion = StructType()
for col in COLS_PUNTUACION:
    esquemaPuntuacion.add(StructField(col[0], col[2], col[3]))
dataframePuntuacion = sqlc.read.format('com.databricks.spark.csv'). \
                option('delimiter', SEPARADOR_TABULADOR). \
                option('header', 'false'). \
                load(RUTA_FICHERO_PUNTUACION, schema=esquemaPuntuacion)
cabecerasReducidas = list()

# Número de NAN y NULL en rating
print('Número de filas con NAN en las puntuaciones en la columna rating: {0}', dataframePuntuacion.where(isnan(COL_PUNTUACION_RATING[0])).count())
print('Número de filas con NULL en las puntuaciones en la columna rating: {0}', dataframePuntuacion.where(isnull(COL_PUNTUACION_RATING[0])).count())

for col in COLS_PUNTUACION:
    if col[0] != COL_PUNTUACION_TIMESTAMP[0]:
        cabecerasReducidas.append(col[0])
dataframePuntuacion = dataframePuntuacion.select(cabecerasReducidas)

# Se carga los usuarios utilizando COLS_USUARIO
esquemaUsuario = StructType()
for col in COLS_USUARIO:
    esquemaUsuario.add(StructField(col[0], col[2], col[3]))
dataframeUsuario = sqlc.read.format('com.databricks.spark.csv'). \
                option('delimiter', SEPARADOR_PIPE). \
                option('header', 'false'). \
                load(RUTA_FICHERO_USUARIO, schema=esquemaUsuario)

# Se carga los items utilizando COLS_ITEM y COLS_ITEM_TODOSGENEROS
esquemaItem = StructType()
for col in COLS_ITEM:
    esquemaItem.add(StructField(col[0], col[2], col[3]))
for col in COLS_ITEM_TODOSGENEROS:
    esquemaItem.add(StructField(col[0], col[2], col[3]))
dataframeItem = sqlc.read.format('com.databricks.spark.csv'). \
                option('delimiter', SEPARADOR_PIPE). \
                option('header', 'false'). \
                load(RUTA_FICHERO_ITEM, schema=esquemaItem)
cabecerasReducidas = list()
for col in COLS_ITEM:
    if col[0] == COL_ITEM_ID[0] or col[0] == COL_ITEM_TITLE[0] or col[0] == COL_ITEM_RELEASEDATE[0]:
        cabecerasReducidas.append(col[0])
for col in COLS_ITEM_TODOSGENEROS:
    cabecerasReducidas.append(col[0])
dataframeItem = dataframeItem.select(cabecerasReducidas)

# Se carga los géneros utilizando COLS_GENERO
esquemaGenero = StructType()
for col in COLS_GENERO:
    esquemaGenero.add(StructField(col[0], col[2], col[3]))
dataframeGenero = sqlc.read.format('com.databricks.spark.csv'). \
                option('delimiter', SEPARADOR_PIPE). \
                option('header', 'false'). \
                load(RUTA_FICHERO_GENERO, schema=esquemaGenero)

print(dataframePuntuacion.show(4))
print(dataframeUsuario.show(4))
print(dataframeItem.show(4))
print(dataframeGenero.show(4))

Número de filas con NAN en las puntuaciones en la columna rating: {0} 0
Número de filas con NULL en las puntuaciones en la columna rating: {0} 0
+------+------+------+
userid|itemid|rating|
+------+------+------+
 196| 242| 3.0|
 186| 302| 3.0|
 22| 377| 1.0|
 244| 51| 2.0|
+------+------+------+
only showing top 4 rows

None
+-----------+---+------+----------+-------+
user_userid|age|gender|occupation|zipcode|
+-----------+---+------+----------+-------+
 1| 24| M|technician| 85711|
 2| 53| F| other| 94043|
 3| 23| M| writer| 32067|
 4| 24| M|technician| 43537|
+-----------+---+------+----------+-------+
only showing top 4 rows

None
+-----------+-----------------+-----------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
item_itemid| title|releasedate|unknown|action|adventure|animation|childrens|comedy|crime|documentary|drama|fantasy|filmnoir|horror|musical|mystery|Romance|scifi|thriller|war|western|
+-----------+-----------------+-----------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
 1| Toy Story (1995)|01-Jan-1995| 0| 0| 0| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 2| GoldenEye (1995)|01-Jan-1995| 0| 1| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0|
 3|Four Rooms (1995)|01-Jan-1995| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0|
 4|Get Shorty (1995)|01-Jan-1995| 0| 1| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----------+-----------------+-----------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
only showing top 4 rows

None
+---------+-------------+
 name|genre_genreid|
+---------+-------------+
 unknown| 0|
 Action| 1|
Adventure| 2|
Animation| 3|
+---------+-------------+
only showing top 4 rows

None

Creamos el Dataframe completo, basado en los dataframes anteriores a través de las claves, y se divide en Train y Test el dataframe a fin de preparar los datos para el aprendizaje automático, con la división estandar 70/30.

Se utiliza el dataframe completo por la razón de que ofrece una visión mucho más amplia al visualizarlo, pero principalmente porque se obtendrán mejores resultados que con el resto de dataframes, principalmente a que se poseen más datos. Al fin y al cabo, el uso del método de factorización de matriz para matrices de calificación se nutre de la mayor cantidad de productos posibles (long tail), y será de ese modo como podamos obtener un modelo más completo.

Si se tratase de un trabajo más específico, y estuviese orientado a un fin concreto o linea de negocio, se suprimirían los atributos que no aportasen valor, y de esta manera aumentaríamos la eficiencia en la ejecución del programa, así como la capacidad del dataframe de soportar cambios o adiciones de atributos, pero no lo consideraremos así para este ejercicio. En definitiva, el único problema que observaremos utilizando este dataframe será el largo tiempo necesario para ejecutar el modelo

In [10]:
# Creamos el dataframe completo con un join de los dataframes puntuación-usuario-item:

dataframePuntuacionUsuarioItem = dataframePuntuacion. \
                                  join(dataframeUsuario, dataframePuntuacion.userid==dataframeUsuario.user_userid, 'left_outer'). \
                                  join(dataframeItem, dataframePuntuacion.itemid==dataframeItem.item_itemid, 'left_outer')
# Se eliminan las columnas duplicadas
cabecerasReducidas = list()
for col in dataframePuntuacionUsuarioItem.columns:
    if col != COL_USUARIO_ID[0] and col !=COL_ITEM_ID[0]:
        cabecerasReducidas.append(col)
dataframePuntuacionUsuarioItem = dataframePuntuacionUsuarioItem.select(cabecerasReducidas)

# Comprobamos que el dataframe resultante está conforme a lo proyectado:

print('Columnas del dataframe: {0}', dataframePuntuacionUsuarioItem.columns)
print('Número de filas: {0}', dataframePuntuacionUsuarioItem.count())

# Se divide el dataframe en entrenamiento y test

dataframePuntuacionUsuarioItem.cache()

dataframePuntuacionUsuarioItemDividido = dataframePuntuacionUsuarioItem.randomSplit([0.7, 0.3], 1234)
dataframePuntuacionUsuarioItemEntrenamiento = dataframePuntuacionUsuarioItemDividido[0]
dataframePuntuacionUsuarioItemTest = dataframePuntuacionUsuarioItemDividido[1]

Columnas del dataframe: {0} ['userid', 'itemid', 'rating', 'age', 'gender', 'occupation', 'zipcode', 'title', 'releasedate', 'unknown', 'action', 'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'filmnoir', 'horror', 'musical', 'mystery', 'Romance', 'scifi', 'thriller', 'war', 'western']
Número de filas: {0} 100000

## Implementación del modelo ALS

Para crear el modelo se utilizará CrossValidator, en el que se variarán varios parámetros: rank, maxIter, alpha y regParam, dentro de unos márgenes normalizados y estándares, que tras varios intentos resultan ofrecer el cálculo del modelo ALS sin aumentar excesivamente el consumo de recursos.

La premisa principal que justifica el uso del CrossValidator es que se ha comprobado que, bajo los parámetros escogidos, sólo un 0,20% de las entradas del dataset del test no tienen valor, y eliminarlos (que es nuestra pretensión), bien puede suponerse que no afectaría en absoluto al modelo, y nos garantizaría evitar tener Nan para el cálculo de la Raíz del error cuadrático medio (rmse).

In [12]:
# Evaluador y cross validation
evaluatorRegression = RegressionEvaluator(labelCol=COL_PUNTUACION_RATING[0])

als = ALS(userCol=COL_PUNTUACION_USERID[0], itemCol=COL_PUNTUACION_ITEMID[0], ratingCol=COL_PUNTUACION_RATING[0], coldStartStrategy='drop')
# als = ALS(userCol=COL_PUNTUACION_USERID[0], itemCol=COL_PUNTUACION_ITEMID[0], ratingCol=COL_PUNTUACION_RATING[0])
grid = ParamGridBuilder(). \
        addGrid(als.rank, [5, 10, 15, 20]). \
        addGrid(als.maxIter, [5, 10]). \
        addGrid(als.alpha, [1.0, 2.0]). \
        addGrid(als.regParam, [0.1, 0.5, 1.0]). \
        build()
# grid = ParamGridBuilder().addGrid(als.rank, [5, 20]).build()
crossValidator = CrossValidator(estimator=als, estimatorParamMaps=grid, evaluator=evaluatorRegression, numFolds=2)
crossValidatorModel = crossValidator.fit(dataframePuntuacionUsuarioItemEntrenamiento)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

## Valoración del modelo

Describimos el método de predicción y valoramos que, efectivamente, no existen NaN en el modelo propuesto gracias al CrossValidation aplicado anteriormente.

In [14]:
# Se obtiene la predicción para entrenamiento y test

dataframePuntuacionUsuarioItemEntrenamientoPrediccion = crossValidatorModel.bestModel.transform(dataframePuntuacionUsuarioItemEntrenamiento)
dataframePuntuacionUsuarioItemTestPrediccion = crossValidatorModel.bestModel.transform(dataframePuntuacionUsuarioItemTest)

dataframePuntuacionUsuarioItemEntrenamiento.show(5)

# Se muestra el número de filas y las que son NAN en el conjunto de entrenamiento y de test

print('Conjunto de entrenamiento predicción -> Número de filas: {0}, número con NAN en predicción: {1}'.format(dataframePuntuacionUsuarioItemEntrenamientoPrediccion.count(), dataframePuntuacionUsuarioItemEntrenamientoPrediccion.where(isnan(COL_PUNTUACION_PREDICTION[0])).count()))
print('Conjunto de test predicción -> Número de filas: {0}, número con NAN en predicción: {1}'.format(dataframePuntuacionUsuarioItemTestPrediccion.count(), dataframePuntuacionUsuarioItemTestPrediccion.where(isnan(COL_PUNTUACION_PREDICTION[0])).count()))

# Describe método para rating y prediction

print(dataframePuntuacionUsuarioItemEntrenamientoPrediccion.select(COL_PUNTUACION_RATING[0], COL_PUNTUACION_PREDICTION[0]).describe().show())
print(dataframePuntuacionUsuarioItemTestPrediccion.select(COL_PUNTUACION_RATING[0], COL_PUNTUACION_PREDICTION[0]).describe().show())

+------+------+------+---+------+----------+-------+--------------------+-----------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
userid|itemid|rating|age|gender|occupation|zipcode| title|releasedate|unknown|action|adventure|animation|childrens|comedy|crime|documentary|drama|fantasy|filmnoir|horror|musical|mystery|Romance|scifi|thriller|war|western|
+------+------+------+---+------+----------+-------+--------------------+-----------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
 1| 4| 3.0| 24| M|technician| 85711| Get Shorty (1995)|01-Jan-1995| 0| 1| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 1| 5| 3.0| 24| M|technician| 85711| Copycat (1995)|01-Jan-1995| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0|
 1| 6| 5.0| 24| M|technician| 85711|Shanghai Triad (Y...|01-Jan-1995| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 1| 7| 4.0| 24| M|technician| 85711|Twelve Monkeys (1...|01-Jan-1995| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0|
 1| 8| 1.0| 24| M|technician| 85711| Babe (1995)|01-Jan-1995| 0| 0| 0| 0| 1| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+------+------+------+---+------+----------+-------+--------------------+-----------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
only showing top 5 rows

Conjunto de entrenamiento predicción -> Número de filas: 70126, número con NAN en predicción: 0
Conjunto de test predicción -> Número de filas: 29817, número con NAN en predicción: 0
+-------+------------------+------------------+
summary| rating| prediction|
+-------+------------------+------------------+
 count| 70126| 70126|
 mean|3.5312437612297862|3.4081050159234176|
 stddev|1.1264763411424088|0.7071796967280013|
 min| 1.0| 0.2048777|
 max| 5.0| 5.757818|
+-------+------------------+------------------+

None
+-------+-----------------+------------------+
summary| rating| prediction|
+-------+-----------------+------------------+
 count| 29817| 29817|
 mean|3.528155079317168|3.3988192242592055|
 stddev|1.122772872442184|0.7050247747441143|
 min| 1.0| -0.5108311|
 max| 5.0| 5.7307315|
+-------+-----------------+------------------+

None

**RMSE** para el conjunto de entrenamiento y el de test. Como se puede observar es más pequeño el Train que el Test, como es lógico. Llama la atención que son valores relativamente altos si los consideramos en el rango 0-1.

In [16]:
# Se obtiene el RMSE sobre los dos conjuntos
rmseEntrenamiento = evaluatorRegression.evaluate(dataframePuntuacionUsuarioItemEntrenamientoPrediccion, {evaluatorRegression.metricName: 'rmse'})
rmseTest = evaluatorRegression.evaluate(dataframePuntuacionUsuarioItemTestPrediccion, {evaluatorRegression.metricName: 'rmse'})

print('RMSE en training: {0}'.format(rmseEntrenamiento))
print('RMSE en test: {0}'.format(rmseTest))

RMSE en training: 0.8016410428223212
RMSE en test: 0.9280585419449354

Se observa en un gráfico la comparativa de las puntuaciones reales y según modelo:

* Se observa que, como en cierto modo es lógico, el modelo no se ajusta a la línea oblicua que debería de seguir. Si observáramos los resultados en un boxplots veríamos que la aparente diferencia del modelo continuo predicho que hemos construido y la evolución real(como ya vimos anteriormente, el error del dataset de entrenamiento es menor),  veríamos que la densidad de puntos es mucho mayor en torno a la zona que rodea la recta que los puntos que se dispersan en su vertical. 

** Se podría jugar con los parámetros del modelo ALS y también haciendo una limpieza previa de datos, para eliminar outliers.

In [18]:
# Se crea la lista con la puntuación real y la predicha en Train y Test

xEntrenamiento, yEntrenamiento = list(), list()
for entrenamientoPrediccion in dataframePuntuacionUsuarioItemEntrenamientoPrediccion.collect():
    xEntrenamiento.append(entrenamientoPrediccion[COL_PUNTUACION_RATING[0]])
    yEntrenamiento.append(entrenamientoPrediccion[COL_PUNTUACION_PREDICTION[0]])
    
xTest, yTest = list(), list()
for testPrediccion in dataframePuntuacionUsuarioItemTestPrediccion.collect():
    xTest.append(testPrediccion[COL_PUNTUACION_RATING[0]])
    yTest.append(testPrediccion[COL_PUNTUACION_PREDICTION[0]])

# Ploteamos

plt.clf()
plt.xlim(-1, 6)
plt.ylim(-1, 6)
plt.xlabel('Puntuacion real')
plt.ylabel('Puntuacion segun el modelo')
plt.title('Puntuacion real vs prediccion')

plt.plot([0, 20], [0, 20], 'b')
# Se pasan los datos de entrenamiento y test al gráfico

plt.plot(xTest, yTest, 'ro', label='Test')
plt.legend(loc='lower right')

plt.show()
display()

In [19]:
plt.clf()
plt.xlim(-1, 6)
plt.ylim(-1, 6)
plt.xlabel('Puntuacion real')
plt.ylabel('Puntuacion segun el modelo')
plt.title('Puntuacion real vs prediccion')

plt.plot([0, 20], [0, 20], 'b')
# Se pasan los datos de entrenamiento y test al gráfico
plt.plot(xEntrenamiento, yEntrenamiento, 'go', label='Train')

plt.legend(loc='lower right')

plt.show()
display()

## Análisis de datos

### Género de películas mejor valorados
Se obtendrá un dataframe donde cada fila tendrá los valores item_id-película-género. Por cada película, habrá tantas filas como géneros tenga.

Para ello:
1. Se obtendrá un diccionario con los géneros donde la clave es el id o posición en el fichero de item y el valor será el género.
2. El siguiente paso es añadir una columna con un array con los géneros que tiene cada película. Para ello se utilizará RDDs para posteriormente trasformalo en otro dataframe
3. El último paso es hacer un explode.

In [22]:
def trasformarFilaColumnasgeneroColumageneros(fila, diccionarionGenero, ultimoIndiceNoGenero=1):

    indice = 0
    indiceGenero = 0
    lineaNueva = list()
    generos = list()
    for item in fila:
        if indice>ultimoIndiceNoGenero:
            if item==1:
                genero = diccionarionGenero.get(indiceGenero)
                if genero:
                    generos.append(genero)
            indiceGenero += 1
        else:
            lineaNueva.append(item)
        indice += 1
    lineaNueva.append(generos)
    return lineaNueva
  
# Se crea un diccionario con los géneros donde la clave es la columna id y el valor el nombre del género

diccionarioGenero = dict()
for row in dataframeGenero.collect():
    diccionarioGenero[row.genre_genreid] = row.name
print(diccionarioGenero)

# Se obtiene el nombre de todas las columnas de los géneros en el dataframe de item

columnas = list()
columnas.append(COL_ITEM_ID[0])
columnas.append(COL_ITEM_TITLE[0])
for col in COLS_ITEM_TODOSGENEROS:
    columnas.append(col[0])
    
# Seleccionamos sólo itemid, titulo, generos

dataframeItemGenero = dataframeItem.select(columnas)
rddItemGenero = dataframeItemGenero.rdd.map(lambda fila: trasformarFilaColumnasgeneroColumageneros(fila, diccionarionGenero=diccionarioGenero))

# Se crea el dataframe desde el RDD

columnas = list()
columnas.append(COL_ITEM_ID[0])
columnas.append(COL_ITEM_TITLE[0])
columnas.append(COL_ITEM_GENEROS[0])
dataframeItemGenero = rddItemGenero.toDF(columnas)

# Hacemos 'explode' para distribuir todas las categorias en filas, y posteriormente comparamos el número de filas

dataframeItemGenero = dataframeItemGenero.select(COL_ITEM_ID[0], COL_ITEM_TITLE[0], explode(dataframeItemGenero.generos).alias(COL_ITEM_GENERO[0]))

print('Nº filas antes de explode {0} y después {1}'.format(dataframeItem.count(), dataframeItemGenero.count()))

dataframeItemGenero.cache()

dataframeItemGenero.show(10)

{0: 'unknown', 1: 'Action', 2: 'Adventure', 3: 'Animation', 4: "Children's", 5: 'Comedy', 6: 'Crime', 7: 'Documentary', 8: 'Drama', 9: 'Fantasy', 10: 'Film-Noir', 11: 'Horror', 12: 'Musical', 13: 'Mystery', 14: 'Romance', 15: 'Sci-Fi', 16: 'Thriller', 17: 'War', 18: 'Western'}
Nº filas antes de explode 1682 y después 2893
+-----------+-----------------+----------+
item_itemid| title| genero|
+-----------+-----------------+----------+
 1| Toy Story (1995)| Animation|
 1| Toy Story (1995)|Children's|
 1| Toy Story (1995)| Comedy|
 2| GoldenEye (1995)| Action|
 2| GoldenEye (1995)| Adventure|
 2| GoldenEye (1995)| Thriller|
 3|Four Rooms (1995)| Thriller|
 4|Get Shorty (1995)| Action|
 4|Get Shorty (1995)| Comedy|
 4|Get Shorty (1995)| Drama|
+-----------+-----------------+----------+
only showing top 10 rows

Se transforma el dataframe del Test sustituyendo filas por el dataframe que se acaba de generar

In [24]:
cabecerasReducidas = list()

for col in COLS_PUNTUACION:
    if col[0] != COL_PUNTUACION_TIMESTAMP[0]:
        cabecerasReducidas.append(col[0])
for col in COLS_USUARIO:
    if col[0] != COL_USUARIO_ID[0]:
        cabecerasReducidas.append(col[0])
        
cabecerasReducidas.append(COL_PUNTUACION_PREDICTION[0])

dataframePuntuacionUsuarioItemTestPrediccionGenero = dataframePuntuacionUsuarioItemTestPrediccion.select(cabecerasReducidas)

# Se hace el join con el otro dataframe y se elimina itemid

dataframePuntuacionUsuarioItemTestPrediccionGenero = dataframePuntuacionUsuarioItemTestPrediccionGenero.join(dataframeItemGenero, dataframePuntuacionUsuarioItemTestPrediccionGenero.itemid==dataframeItemGenero.item_itemid, 'inner')

print('Número de filas antes en el conjunto de test antes de hacer el join {0} y después {1}'.format(dataframePuntuacionUsuarioItemTestPrediccion.count(), dataframePuntuacionUsuarioItemTestPrediccionGenero.count()))

dataframeItemGenero.unpersist()
dataframePuntuacionUsuarioItemTestPrediccionGenero.cache()

dataframePuntuacionUsuarioItemTestPrediccionGenero.show(10)

Número de filas antes en el conjunto de test antes de hacer el join 29817 y después 63496
+------+------+------+---+------+-------------+-------+----------+-----------+--------------------+------+
userid|itemid|rating|age|gender| occupation|zipcode|prediction|item_itemid| title|genero|
+------+------+------+---+------+-------------+-------+----------+-----------+--------------------+------+
 243| 26| 3.0| 33| M| educator| 60201| 3.195533| 26|Brothers McMullen...|Comedy|
 593| 26| 4.0| 31| F| educator| 68767| 3.2315865| 26|Brothers McMullen...|Comedy|
 436| 26| 3.0| 30| F|administrator| 17345| 3.5340867| 26|Brothers McMullen...|Comedy|
 406| 26| 3.0| 52| M| educator| 93109| 3.1100733| 26|Brothers McMullen...|Comedy|
 336| 26| 5.0| 23| M| salesman| 42101| 3.3243408| 26|Brothers McMullen...|Comedy|
 707| 26| 3.0| 56| F| librarian| 19146| 3.3416667| 26|Brothers McMullen...|Comedy|
 222| 26| 3.0| 29| M| programmer| 27502| 3.0283246| 26|Brothers McMullen...|Comedy|
 886| 26| 4.0| 20| M| student| 61820| 2.8934784| 26|Brothers McMullen...|Comedy|
 327| 26| 3.0| 22| M| student| 11101| 3.0975115| 26|Brothers McMullen...|Comedy|
 318| 26| 5.0| 65| M| retired| 06518| 3.5815697| 26|Brothers McMullen...|Comedy|
+------+------+------+---+------+-------------+-------+----------+-----------+--------------------+------+
only showing top 10 rows

Valoración de películas por genéro: Para valorar los géneros mejor valorados, vamos a hacer una consulta con las medias de rating y predicción por género

In [26]:
print(dataframePuntuacionUsuarioItemTestPrediccionGenero.groupBy(COL_ITEM_GENERO[0]).agg(avg(COL_PUNTUACION_RATING[0]).alias(COL_PUNTUACION_RATING[0]), avg(COL_PUNTUACION_PREDICTION[0]).alias(COL_PUNTUACION_PREDICTION[0]), count('*').alias(COL_PUNTUACION_NUMEROPUNTUACIONES[0])).sort(COL_PUNTUACION_RATING[0], ascending=False).show())

+-----------+------------------+------------------+-------------------+
 genero| rating| prediction|numero_puntuaciones|
+-----------+------------------+------------------+-------------------+
 Film-Noir| 3.949044585987261|3.7403907603772075| 471|
 War|3.8166318719554626|3.6786669243435606| 2874|
 Drama|3.6908005381769255|3.5557855231906084| 11892|
 Mystery|3.6447368421052633|3.4796821794031483| 1520|
 Crime|3.6419031719532553|3.4846138363091494| 2396|
 Romance| 3.634884534639608|3.5076095221747963| 5716|
Documentary| 3.625|3.5093206932832457| 232|
 Western| 3.613259668508287|3.5009253349971594| 543|
 Animation| 3.552583025830258|3.4494669570474166| 1084|
 Sci-Fi|3.5469627360898417|3.4380775368931706| 3918|
 Musical| 3.535614525139665| 3.406676713835094| 1432|
 Thriller|3.4972434915773354|3.3655292038302327| 6530|
 Action|3.4837329876863254|3.3605296531476068| 7715|
 Adventure|3.4790911288373216| 3.39731672747659| 4137|
 Comedy|3.4054633705835875|3.2607506290052934| 8859|
 Horror|3.3407544836116263| 3.162789779824096| 1617|
 unknown|3.3333333333333335| 3.273937463760376| 3|
 Children's| 3.318625174175569| 3.232146858601475| 2153|
 Fantasy| 3.198019801980198| 3.125619927845379| 404|
+-----------+------------------+------------------+-------------------+

None

Film-noir es el género que tiene mejor valoración, pero hace sospechar el hecho de que es uno de los géneros menos votados, lo que significa que está orientado a un público más exclusivo, y esto podría, según como decíamos en los inicios, no ser un análisis suficiente o concluyente si el fin último fuera aumentar beneficios generando contenido basado en género. 

Dadas esas circunstancias, valoraremos de nuevo el ranking eliminando aquellas cuyo número de puntiaciones esté por debajo de 1000:

In [28]:
dataframePuntuacionUsuarioItemTestPrediccionGeneroGroup= dataframePuntuacionUsuarioItemTestPrediccionGenero.groupBy(COL_ITEM_GENERO[0]).agg(avg(COL_PUNTUACION_RATING[0]).alias(COL_PUNTUACION_RATING[0]), avg(COL_PUNTUACION_PREDICTION[0]).alias(COL_PUNTUACION_PREDICTION[0]), count('*').alias(COL_PUNTUACION_NUMEROPUNTUACIONES[0]))

print(dataframePuntuacionUsuarioItemTestPrediccionGeneroGroup.where(dataframePuntuacionUsuarioItemTestPrediccionGeneroGroup.numero_puntuaciones>1000).sort(COL_PUNTUACION_RATING[0], ascending=False).show())

+----------+------------------+------------------+-------------------+
 genero| rating| prediction|numero_puntuaciones|
+----------+------------------+------------------+-------------------+
 War|3.8166318719554626|3.6786669243435606| 2874|
 Drama|3.6908005381769255|3.5557855231906084| 11892|
 Mystery|3.6447368421052633|3.4796821794031483| 1520|
 Crime|3.6419031719532553|3.4846138363091494| 2396|
 Romance| 3.634884534639608|3.5076095221747963| 5716|
 Animation| 3.552583025830258|3.4494669570474166| 1084|
 Sci-Fi|3.5469627360898417|3.4380775368931706| 3918|
 Musical| 3.535614525139665| 3.406676713835094| 1432|
 Thriller|3.4972434915773354|3.3655292038302327| 6530|
 Action|3.4837329876863254|3.3605296531476068| 7715|
 Adventure|3.4790911288373216| 3.39731672747659| 4137|
 Comedy|3.4054633705835875|3.2607506290052934| 8859|
 Horror|3.3407544836116263| 3.162789779824096| 1617|
Children's| 3.318625174175569| 3.232146858601475| 2153|
+----------+------------------+------------------+-------------------+

None

Y como se puede comprobar, es el género de guerra quien gana en mayor rating. 

Se podría hacer también una agrupación por **género y código postal** para ver si en una zona predomina las buenas puntuaciones de un determinado género, filtrando aquellas que tengan un número mínimo de puntuaciones mayor que 50.

In [30]:
dataframePuntuacionUsuarioItemTestPrediccionGeneroGroup = dataframePuntuacionUsuarioItemTestPrediccionGenero.groupBy([COL_ITEM_GENERO[0], COL_USUARIO_ZIPCODE[0]]).agg(avg(COL_PUNTUACION_RATING[0]).alias(COL_PUNTUACION_RATING[0]), avg(COL_PUNTUACION_PREDICTION[0]).alias(COL_PUNTUACION_PREDICTION[0]), count('*').alias(COL_PUNTUACION_NUMEROPUNTUACIONES[0]))

print(dataframePuntuacionUsuarioItemTestPrediccionGeneroGroup.where(dataframePuntuacionUsuarioItemTestPrediccionGeneroGroup.numero_puntuaciones>50).sort(COL_PUNTUACION_RATING[0], ascending=False).show())

+--------+-------+------------------+------------------+-------------------+
 genero|zipcode| rating| prediction|numero_puntuaciones|
+--------+-------+------------------+------------------+-------------------+
 Drama| 97520| 4.134615384615385|3.9813398523972583| 52|
 Drama| 92626| 4.109589041095891| 3.82917049812944| 73|
 Drama| 11758|3.9863013698630136|3.8211512957533746| 73|
 Drama| 48197|3.9607843137254903| 3.637568908579209| 51|
 Drama| 02215|3.9019607843137254|3.6423270982854508| 51|
 Drama| 14216|3.8679245283018866| 3.581475055442666| 53|
 Drama| 48103| 3.8625| 3.620722621679306| 80|
 Drama| 83702|3.7857142857142856|3.7061876910073415| 56|
 Drama| 55414| 3.763358778625954|3.5315960591075983| 131|
 Drama| 80525|3.7209302325581395|3.5106907278992407| 86|
 Action| 61820|3.6538461538461537|3.4510942628750434| 52|
 Drama| 29206|3.6527777777777777|3.3148333215051227| 72|
 Drama| 10003|3.6464646464646466| 3.571312640652512| 99|
 Comedy| 48103|3.6444444444444444| 3.401814309755961| 90|
 Romance| 55414| 3.641509433962264|3.3802655193040954| 53|
 Drama| 20009| 3.64|3.5963089799880983| 100|
Thriller| 55414| 3.617283950617284|3.3895504003689614| 81|
 Drama| 61820| 3.616279069767442| 3.538461496663648| 86|
 Action| 55414|3.5795454545454546| 3.296123272993348| 88|
 Comedy| 80525| 3.566265060240964|3.3144876956939697| 83|
+--------+-------+------------------+------------------+-------------------+
only showing top 20 rows

None

### Películas mejor valoradas
Al igual que con el género, se valoran el número de películas mejor valoradas por título:

In [32]:
dataframePuntuacionUsuarioItemTestPrediccionGroup = dataframePuntuacionUsuarioItemTestPrediccion.groupBy(COL_ITEM_TITLE[0]).agg(avg(COL_PUNTUACION_RATING[0]).alias(COL_PUNTUACION_RATING[0]), avg(COL_PUNTUACION_PREDICTION[0]).alias(COL_PUNTUACION_PREDICTION[0]), count('*').alias(COL_PUNTUACION_NUMEROPUNTUACIONES[0]))
print(dataframePuntuacionUsuarioItemTestPrediccionGroup.where(dataframePuntuacionUsuarioItemTestPrediccionGroup.numero_puntuaciones>60).sort(COL_PUNTUACION_RATING[0], ascending=False).show())

+--------------------+------------------+------------------+-------------------+
 title| rating| prediction|numero_puntuaciones|
+--------------------+------------------+------------------+-------------------+
Schindler's List ...| 4.455555555555556| 4.278086402681139| 90|
 Casablanca (1942)| 4.436619718309859| 4.346449049425797| 71|
 Star Wars (1977)| 4.427807486631016| 4.224509615311648| 187|
One Flew Over the...| 4.414285714285715| 4.1923647914614| 70|
Shawshank Redempt...| 4.411111111111111| 4.177539041307237| 90|
Godfather, The (1...| 4.325581395348837| 4.02985154381094| 129|
Dr. Strangelove o...| 4.311475409836065| 4.157566692008347| 61|
Usual Suspects, T...| 4.305882352941176| 4.189605886795942| 85|
 Braveheart (1995)| 4.301204819277109| 4.018423089061875| 83|
Godfather: Part I...| 4.27027027027027|4.0521568672077075| 74|
Good Will Hunting...| 4.26984126984127| 4.205939058273557| 63|
 Titanic (1997)| 4.262135922330097| 4.061602395715066| 103|
 Amadeus (1984)| 4.252747252747253| 4.013721057346889| 91|
Silence of the La...| 4.224| 4.120173648834228| 125|
Princess Bride, T...| 4.212765957446808| 4.033322428135162| 94|
Monty Python and ...| 4.206521739130435|3.8568470776081085| 92|
 Alien (1979)| 4.197530864197531| 3.879824123264831| 81|
Wizard of Oz, The...| 4.191780821917808| 3.944918100148031| 73|
To Kill a Mocking...|4.1911764705882355| 4.101320687462302| 68|
Empire Strikes Ba...| 4.177570093457944| 4.066192127833856| 107|
+--------------------+------------------+------------------+-------------------+
only showing top 20 rows

None

La lista de Schindler, Casablanca, Star Wars, Alguien voló sobre el nido del cuco.... 

Pese a que el análisis podría complejizarse mucho más, lo interesante es que vemos que la pridicción se ajusta bastante bien a los resultados por agrupación, aunque un poco por debajo